##### This script validates SOCP/test_model.ipynb

In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use({'figure.facecolor':'white'})

In [2]:
def read_input(filename):
    SystemDemand = pd.read_excel(filename, sheet_name = 'SystemDemand')
    NodeData = pd.read_excel(filename, sheet_name='NodeData')
    LineData = pd.read_excel(filename, sheet_name='LineData')

    return {'SystemDemand':SystemDemand, 'NodeData':NodeData, 'LineData':LineData}


In [3]:
data = read_input(r'Input_Files\InputData34.xlsx')

def optimization_model(inputData, Vmax, Vmin, Vnom, Sbase, Zbase):

    SystemDemand= inputData['SystemDemand']
    NodeData = inputData['NodeData']
    LineData = inputData['LineData']
    time = [SystemDemand.loc[i, 'TIME'] for i in SystemDemand.index]
    lines = [(LineData.loc[i, 'FROM'], LineData.loc[i, 'TO']) for i in LineData.index]
    buses = [NodeData.loc[i, 'NODES'] for i in NodeData.index]
    R = {(LineData.loc[i,'FROM'],LineData.loc[i,'TO']):LineData.loc[i,'R'] for i in LineData.index}
    X = {(LineData.loc[i,'FROM'],LineData.loc[i,'TO']):LineData.loc[i,'X'] for i in LineData.index}
    B = {(LineData.loc[i,'FROM'],LineData.loc[i,'TO']):LineData.loc[i,'B'] for i in LineData.index}
    Tb = {buses[i]:NodeData.loc[i, 'Tb'] for i in NodeData.index}
    Pd = {(buses[i], time[k]):NodeData.loc[i, 'PD']*SystemDemand.loc[k, 'PD']/Sbase for k in SystemDemand.index for i in NodeData.index}
    Qd = {(buses[i], time[k]):NodeData.loc[i, 'QD']*SystemDemand.loc[k, 'QD']/Sbase for k in SystemDemand.index for i in NodeData.index}

    #---------------------------------------------------------------------------------------------------------
    #Define the Model
    #---------------------------------------------------------------------------------------------------------

    model = ConcreteModel()

    #---------------------------------------------------------------------------------------------------------
    #Define Sets
    #---------------------------------------------------------------------------------------------------------
    model.LINES = Set(initialize=lines)
    model.NODES = Set(initialize=buses)
    model.TIME = Set(ordered=True, initialize=time)
    #---------------------------------------------------------------------------------------------------------
    #Define Parameters
    #---------------------------------------------------------------------------------------------------------

    model.R = Param(model.LINES, initialize=R, mutable=True, within=NonNegativeReals)
    model.X = Param(model.LINES, initialize=X, mutable=True, within=NonNegativeReals)
    model.B = Param(model.LINES, initialize=B, mutable=True, within=NonNegativeReals)

    model.Pd = Param(model.NODES, model.TIME, initialize=Pd, mutable=True, within=Any)
    model.Qd = Param(model.NODES, model.TIME, initialize=Qd, mutable=True, within=Any)
    model.Tb = Param(model.NODES, initialize=Tb, mutable=True, within=Any)
    model.Vmax = Param(initialize=Vmax, mutable=True) # p.u.
    model.Vmin = Param(initialize=Vmin, mutable=True) # p.u.
    model.Vnom = Param(initialize=1.0, mutable=True)  # p.u.

    #---------------------------------------------------------------------------------------------------------
    #Initialize Parameters
    #---------------------------------------------------------------------------------------------------------

    def ini_pu_impedance_r(model, i,j):
        return model.R[i,j]/Zbase
    model.rij = Param(model.LINES, rule=ini_pu_impedance_r)                        # p.u.
    def ini_pu_impedance_x(model, i,j):
        return model.X[i,j]/Zbase
    model.xij = Param(model.LINES, rule=ini_pu_impedance_x)                        # p.u.
    def ini_pu_g_series(model, i,j):
        return (model.R[i,j]/(model.R[i,j]**2 + model.X[i,j]**2))/(1/Zbase)
    model.gij = Param(model.LINES, rule=ini_pu_g_series)                           # p.u.
    def ini_pu_b_series(model, i,j):
        return -(model.X[i,j]/(model.R[i,j]**2 + model.X[i,j]**2))/(1/Zbase)
    model.bij = Param(model.LINES, rule=ini_pu_b_series)                           # p.u.
    def ini_pu_b_shunt(model, i,j):
        return model.B[i,j]/2
    model.bsh = Param(model.LINES, rule=ini_pu_b_shunt)                            # p.u.

    #---------------------------------------------------------------------------------------------------------
    #Define Variables
    #---------------------------------------------------------------------------------------------------------

    model.Pde = Var(model.LINES, model.TIME, initialize=0)
    model.Qde = Var(model.LINES, model.TIME, initialize=0)
    model.Ppa = Var(model.LINES, model.TIME, initialize=0)
    model.Qpa = Var(model.LINES, model.TIME, initialize=0)

    def active_supply_rule(model, n, t):
        if model.Tb[n] == 0:
            temp = 0.0
            model.Ps[n,t].fixed = True
        else:
            temp = 0.0
            model.Ps[n,t].fixed = False
        return temp
    model.Ps = Var(model.NODES, model.TIME, initialize=active_supply_rule)

    def reactive_supply_rule(model, n, t):
        if model.Tb[n] == 0:
            temp = 0.0
            model.Qs[n,t].fixed = True
        else:
            temp = 0.0
            model.Qs[n,t].fixed = False
        return temp
    model.Qs = Var(model.NODES, model.TIME, initialize=reactive_supply_rule)

    def ini_voltage(model, n, t):
        if model.Tb[n] == 0:
            temp = model.Vnom
            model.V[n,t].fixed = False
        else:
            temp = model.Vnom
            model.V[n,t].fixed = True
        return temp
    model.V = Var(model.NODES, model.TIME, initialize=ini_voltage)

    def ini_voltage_angle(model, n, t):
        if model.Tb[n] == 0:
            temp = 0
            model.delta[n,t].fixed = False
        else:
            delta_slack = 0
            temp = delta_slack
            model.delta[n,t].fixed = True
        return temp
    model.delta = Var(model.NODES, model.TIME, initialize=ini_voltage_angle)

    #---------------------------------------------------------------------------------------------------------
    #Define Objective
    #---------------------------------------------------------------------------------------------------------

    def act_loss(model):
        return sum(model.Pde[i,j,t] + model.Ppa[i,j,t] for i,j in model.LINES for t in model.TIME)
    model.obj = Objective(rule=act_loss)

    #---------------------------------------------------------------------------------------------------------
    #Define Constraints
    #---------------------------------------------------------------------------------------------------------

    def active_power_flow_rule(model, k, t):
        return model.Ps[k,t] - model.Pd[k,t] - sum(model.Pde[i,j,t] for i,j in model.LINES if i == k) - \
               sum(model.Ppa[i,j,t] for i,j in model.LINES if j == k) == 0
    model.active_power_flow = Constraint(model.NODES, model.TIME, rule=active_power_flow_rule)

    def reactive_power_flow_rule(model, k, t):
     return model.Qs[k,t] - model.Qd[k,t] - sum(model.Qde[i,j,t] for i,j in model.LINES if i == k) - \
            sum(model.Qpa[i,j,t] for i,j in model.LINES if j == k) == 0
    model.reactive_power_flow = Constraint(model.NODES, model.TIME, rule=reactive_power_flow_rule)

    def active_power_rule_de(model, i,j, t):
        return model.Pde[i,j,t] - model.gij[i,j]*model.V[i,t]**2 + \
               model.V[i,t]*model.V[j,t]*model.gij[i,j]*cos(model.delta[i,t] - model.delta[j,t]) + \
               model.V[i,t]*model.V[j,t]*model.bij[i,j]*sin(model.delta[i,t] - model.delta[j,t]) == 0
    model.active_power_out = Constraint(model.LINES, model.TIME, rule=active_power_rule_de)

    def reactive_power_rule_de(model, i,j, t):
        return model.Qde[i,j,t] + (model.bij[i,j] + model.bsh[i,j])*model.V[i,t]**2 + \
               model.V[i,t]*model.V[j,t]*model.gij[i,j]*sin(model.delta[i,t] - model.delta[j,t]) - \
               model.V[i,t]*model.V[j,t]*model.bij[i,j]*cos(model.delta[i,t] - model.delta[j,t]) == 0
    model.reactive_power_out = Constraint(model.LINES, model.TIME, rule=reactive_power_rule_de)

    def active_power_rule_pa(model, i,j, t):
        return model.Ppa[i,j,t] - model.gij[i,j]*model.V[j,t]**2 + \
               model.V[i,t]*model.V[j,t]*model.gij[i,j]*cos(model.delta[i,t] - model.delta[j,t]) - \
               model.V[i,t]*model.V[j,t]*model.bij[i,j]*sin(model.delta[i,t] - model.delta[j,t]) == 0
    model.active_power_in = Constraint(model.LINES, model.TIME, rule=active_power_rule_pa)

    def reactive_power_rule_pa(model, i,j, t):
        return model.Qpa[i,j,t] + (model.bij[i,j] + model.bsh[i,j])*model.V[j,t]**2 - \
               model.V[i,t]*model.V[j,t]*model.gij[i,j]*sin(model.delta[i,t] - model.delta[j,t]) - \
               model.V[i,t]*model.V[j,t]*model.bij[i,j]*cos(model.delta[i,t] - model.delta[j,t]) == 0
    model.reactive_power_in = Constraint(model.LINES, model.TIME, rule=reactive_power_rule_pa)

    def voltage_limit_rule(model, n, t):
        return inequality(model.Vmin, model.V[n,t], model.Vmax)
    model.voltage_limit = Constraint(model.NODES, model.TIME, rule=voltage_limit_rule)

    return model, buses

In [4]:
Sbase = 1000 # kVA
Vnom = 11/np.sqrt(3)    # kV
Zbase = (Vnom**2)*1000/Sbase # Ohm
Vmax = 1.05
Vmin = 0.7
model, buses = optimization_model(data, Vmax, Vmin, Vnom, Sbase, Zbase)
#model.pprint()

In [9]:
solver = SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    18864
Number of nonzeros in inequality constraint Jacobian.:      792
Number of nonzeros in Lagrangian Hessian.............:     5448

Total number of variables............................:     4800
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 5616, 'Number of variables': 4800, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.13\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.8664581775665283}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
losses = value(model.obj)
supply = sum([sum(value(model.Ps[n, t]) for n in model.NODES) for t in model.TIME])
demand = sum([sum(value(model.Pd[n, t]) for n in model.NODES) for t in model.TIME])
print([supply, demand, losses])
print([supply*Sbase, demand*Sbase, losses*Sbase, demand*Sbase + losses*Sbase])

In [ ]:
# DATA VISUALIZATION - VOLTAGES AND ACTIVE/REACTIVE POWERS AND CURRENTS/LOADING OF LINES
voltage = pd.DataFrame(columns=buses, index=data['SystemDemand']['TIME'].values)
active_d = pd.DataFrame(columns=buses, index=data['SystemDemand']['TIME'].values)
reactive_d = pd.DataFrame(columns=buses, index=data['SystemDemand']['TIME'].values)
active_s = pd.DataFrame(columns=buses, index=data['SystemDemand']['TIME'].values)
reactive_s = pd.DataFrame(columns=buses, index=data['SystemDemand']['TIME'].values)

for t in model.TIME:
    for n in model.NODES:
        voltage.loc[t,n] = model.V[n,t].value
        active_d.loc[t,n] = model.Pd[n,t].value*Sbase
        reactive_d.loc[t,n] = model.Qd[n,t].value*Sbase
        active_s.loc[t,n] = model.Ps[n,t].value*Sbase
        reactive_s.loc[t,n] = model.Qs[n,t].value*Sbase

fig_pow, ax_pow = plt.subplots(nrows=2, ncols=2, squeeze=False, figsize=(12,8))
ax_pow[0,0].plot(active_d.index, active_d, marker='o');ax_pow[0,0].set_xlabel('Time [h]');ax_pow[0,0].set_ylabel('Active Power [kW]');ax_pow[0,0].set_title('Active Power Demand');
ax_pow[0,1].plot(reactive_d.index, reactive_d, marker='o');ax_pow[0,1].set_xlabel('Time [h]');ax_pow[0,1].set_ylabel('Reactive Power [kVar]');ax_pow[0,1].set_title('Reactive Power Demand');
ax_pow[1,0].plot(active_s.index, active_s, marker='o');ax_pow[1,0].set_xlabel('Time [h]');ax_pow[1,0].set_ylabel('Active Power [kW]');ax_pow[1,0].set_title('Active Power Supply');
ax_pow[1,1].plot(reactive_s.index, reactive_s, marker='o');ax_pow[1,1].set_xlabel('Time [h]');ax_pow[1,1].set_ylabel('Reactive Power [kVar]');ax_pow[1,1].set_title('Reactive Power Supply');
fig_pow.legend(['Bus'+str(i) for i in active_d.columns], bbox_to_anchor=(1.04,0.5), loc='center', borderaxespad=0);
fig_pow.tight_layout()
fig_pow.show;
plt.savefig('power.png', bbox_inches='tight')

b=['Bus'+str(i) for i in active_d.columns]
b.extend({'$V_{min}$','$V_{max}$'})
fig_v, ax_v = plt.subplots(nrows=1, ncols=1, squeeze=False, figsize=(12,8))
ax_v[0,0].plot(voltage.index, voltage, marker='o');ax_v[0,0].set_xlabel('Time [h]');ax_v[0,0].set_ylabel('Voltage [p.u.]');ax_v[0,0].set_title('Voltage');
ax_v[0,0].axhline(model.Vmin.value, color='r', linestyle='--')
ax_v[0,0].axhline(model.Vmax.value, color='r', linestyle='--')
ax_v[0,0].legend(b, bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
fig_v.tight_layout()
fig_v.show;
plt.savefig('voltage.png', bbox_inches='tight')